<a href="https://colab.research.google.com/github/BucketofJava/EnglishToPhonetics/blob/main/EnglishToPhonetics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#English to Phonetics

####Imports

In [ ]:
import urllib.request as URL
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.sequence as kps
from torch import nn;
from torch import Tensor;
from torch import optim;
from torch.utils.data import TensorDataset, DataLoader;

import torch.cuda


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

##Data Processing

####Fetching Data

In [ ]:
dictionary_url="https://github.com/cmusphinx/cmudict/raw/master/"
dictionary_fileName="cmudict.dict"
URL.urlretrieve(dictionary_url+dictionary_fileName, dictionary_fileName)
dict_file="/content/"+dictionary_fileName

In [ ]:
f=open(dict_file, "r")
full_text=f.read()
full_text_split=full_text.split("\n")
word_list=[]
phonetics_list=[]
for i in range(len(full_text_split)):
  full_text_split[i]=full_text_split[i].split(" ")
  word_list.append(list(full_text_split[i][0]))
  phonetics_list.append(full_text_split[i][1:])
print(full_text_split[:1000])

[["'bout", 'B', 'AW1', 'T'], ["'cause", 'K', 'AH0', 'Z'], ["'course", 'K', 'AO1', 'R', 'S'], ["'cuse", 'K', 'Y', 'UW1', 'Z'], ["'em", 'AH0', 'M'], ["'frisco", 'F', 'R', 'IH1', 'S', 'K', 'OW0'], ["'gain", 'G', 'EH1', 'N'], ["'kay", 'K', 'EY1'], ["'m", 'AH0', 'M'], ["'n", 'AH0', 'N'], ["'round", 'R', 'AW1', 'N', 'D'], ["'s", 'EH1', 'S'], ["'til", 'T', 'IH1', 'L'], ["'tis", 'T', 'IH1', 'Z'], ["'twas", 'T', 'W', 'AH1', 'Z'], ['a', 'AH0'], ['a(2)', 'EY1'], ["a's", 'EY1', 'Z'], ['a.', 'EY1'], ["a.'s", 'EY1', 'Z'], ['a.d.', 'EY2', 'D', 'IY1'], ['a.m.', 'EY2', 'EH1', 'M'], ['a.s', 'EY1', 'Z'], ['aaa', 'T', 'R', 'IH2', 'P', 'AH0', 'L', 'EY1'], ['aaberg', 'AA1', 'B', 'ER0', 'G'], ['aachen', 'AA1', 'K', 'AH0', 'N'], ['aachener', 'AA1', 'K', 'AH0', 'N', 'ER0'], ['aaker', 'AA1', 'K', 'ER0'], ['aalborg', 'AO1', 'L', 'B', 'AO0', 'R', 'G', '#', 'place,', 'danish'], ['aalborg(2)', 'AA1', 'L', 'B', 'AO0', 'R', 'G'], ['aalburg', 'AE1', 'L', 'B', 'ER0', 'G', '#', 'place,', 'dutch'], ['aalen', 'AE1', 'L', 

###Tokenize phonetics & letters

In [ ]:
letter_tokenizer=Tokenizer()

flattened_wordlist=[character for word in word_list for character in word]
letter_tokenizer.fit_on_texts(flattened_wordlist)
tokenized_words=letter_tokenizer.texts_to_sequences(word_list)
tokenized_words=kps.data_utils.pad_sequences(tokenized_words)[:]

In [ ]:
phonetic_tokenizer=Tokenizer()
flattened_phoneticlist=[phoneme for word in phonetics_list for phoneme in word]
phonetic_tokenizer.fit_on_texts(flattened_phoneticlist)
tokenized_phonetics=phonetic_tokenizer.texts_to_sequences(phonetics_list)
tokenized_phonetics=kps.data_utils.pad_sequences(tokenized_phonetics)[:]

In [ ]:
print(tokenized_phonetics.shape)

(135156, 28)


In [ ]:
phonetic_length=tokenized_phonetics.shape[1]
word_length=tokenized_words.shape[1]

In [ ]:
tokenized_phonetics=tokenized_phonetics.astype('float32')
tokenized_words=tokenized_words.reshape(tokenized_words.shape[0], tokenized_words.shape[1], 1).astype('float32')


In [ ]:
indices=np.random.shuffle(np.array(range(tokenized_words.shape[0])))
tokenized_phonetics=tokenized_phonetics[indices]
tokenized_words=tokenized_words[indices]

In [ ]:
tokenized_words=tokenized_words[0]
tokenized_phonetics=tokenized_phonetics[0]

In [ ]:
split_index=int(tokenized_words.shape[0]*0.8)

words_train, words_test=(tokenized_words[:split_index], tokenized_words[split_index:])
phonetics_train, phonetics_test=(tokenized_phonetics[:split_index], tokenized_phonetics[split_index:])


In [ ]:
print(words_train.shape)

(108124, 28, 1)


In [ ]:
words_train_tensor, words_test_tensor=(Tensor(words_train), Tensor(words_test))
phonetics_train_tensor, phonetics_test_tensor=(Tensor(phonetics_train), Tensor(phonetics_test))

In [ ]:
print(words_train_tensor)

tensor([[[ 0.],
         [ 0.],
         [ 0.],
         ...,
         [ 7.],
         [13.],
         [ 8.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ...,
         [13.],
         [ 5.],
         [ 1.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ...,
         [ 3.],
         [ 5.],
         [ 1.]],

        ...,

        [[ 0.],
         [ 0.],
         [ 0.],
         ...,
         [14.],
         [ 1.],
         [ 5.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ...,
         [ 4.],
         [ 6.],
         [15.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         ...,
         [15.],
         [14.],
         [ 8.]]])


In [ ]:
etp_train_dataset=TensorDataset(words_train_tensor.long(), phonetics_train_tensor)
etp_train_dataloader=DataLoader(etp_train_dataset, batch_size=1)
etp_test_dataset=TensorDataset(words_test_tensor.long(), phonetics_test_tensor)
etp_test_dataloader=DataLoader(etp_test_dataset, batch_size=1)

## Model Definition and Training

In [ ]:
class ETPModel(nn.Module):
  def __init__(self, num_word_chars, num_phonetic_chars, embedding_size, hidden_state_size, num_encoder_layers):
    super().__init__()
    self.encoder=nn.Sequential(
        #Embedding layer: Tokens -> Vectors
        nn.Embedding(31, embedding_size),
        #LSTM layer(s): Vectorized Tokens -> Hidden and Output states
        nn.LSTM(input_size=embedding_size, hidden_size=hidden_state_size, num_layers=num_encoder_layers)
    );
    #self.decoder=nn.Sequential()
  def forward(self, x):
    return self.encoder(x)
  def train_model(self, dataloader, loss_func, optimizer):
    self.train();
    for batch, (x, y) in enumerate(dataloader):
      prediction=self(x)
      loss=loss_func(prediction, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    




In [ ]:
etpmodel=ETPModel(len(letter_tokenizer.word_index)+1, len(phonetic_tokenizer.word_index)+1, 32, 8, 1).to(device)
etpmodel.train_model(etp_train_dataloader, nn.CrossEntropyLoss(), optim.Adam(etpmodel.parameters(), lr=0.01))

RuntimeError: ignored

In [ ]:
print(len(list(enumerate(etp_train_dataloader))))

108124


In [ ]:
for batch, (x, y) in enumerate(etp_train_dataloader):
  print(etpmodel(x))

Streaming output truncated to the last 5000 lines.
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-7.9876e-02,  2.6730e-01],
         [-4.1331e-02,  3.6081e-01],
         [-6.0319e-02,  3.1737e-01],
         [-5.2947e-02,  3.3161e-01],
         [-4.8476e-02,  3.4202e-01],
         [-3.9485e-02,  3.6575e-01],
         [-9.0335e-02,  2.0131e-01]],

        [[ 2.9087e-02, -2.3955e-01],
         [ 2.9087e-02, -2.3955e-01],
         [ 2.9087e-02, -2.3955e-01],
         [ 2.9087e-02, -2.3955e-01],
         [ 2.9087e-02,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.0799,  0.2673],
         [-0.0457,  0.3492],
         [-0.0457,  0.3492],
         [-0.0655,  0.3086],
         [-0.0469,  0.3459],
         [-0.0504,  0.3373],
         [-0.0903,  0.2013],
         [-0.0457,  0.3492]],

        [[ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-1.0146e-01,  1.1439e-01],
         [-7.9876e-02,  2.6730e-01],
         [-4.0114e-02,  3.6407e-01],
         [-6.0319e-02,  3.1737e-01],
         [-4.3043e-02,  3.5621e-01],
         [-7.9876e-02,  2.6730e-01],
         [-7.1866e-02,  2.9575e-01],
         [-3.8967e-02,  3.6714e-01],
         [-6.0319e-02,  3.1737e-01]],

        [[ 2.9087e-02, -2.3955e-01],
         [ 2.9087e-02, -2.3955e-01],
         [ 2.9087e-02, -2.3955e-01],
         [ 2.9087e-02, -2.3955e-01],
         [ 2.9087e-02,

...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.0799,  0.2673],
         [-0.0562,  0.3250],
         [-0.0655,  0.3086],
         [-0.0446,  0.3519],
         [-0.0655,  0.3086],
         [-0.0603,  0.3174],
         [-0.0504,  0.3373]],

        [[ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.1015,  0.1144],
         [-0.0799,  0.2673],
         [-0.0562,  0.3250],
         [-0.0504,  0.3373],
         [-0.0438,  0.3542],
         [-0.0562,  0.3250],
         [-0.0903,  0.2013],
         [-0.0603,  0.3174]],

        [[ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, -0.2395],
         [ 0.0291, 

KeyboardInterrupt: ignored

In [ ]:

etpencodermodel=Sequential()
#tokenized_phonetics=tokenized_phonetics.reshape(tokenized_phonetics.shape[0], tokenized_phonetics.shape[1], 1).astype('float32')

print(tokenized_words[:1000])
etpmodel.add(Bidirectional(LSTM(word_length, return_sequences=True)))
etpmodel.add(Bidirectional(LSTM(2, return_sequences=True)))

etpmodel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['acc'])

[[[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 7.]
  [13.]
  [ 8.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [13.]
  [ 5.]
  [ 1.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 3.]
  [ 5.]
  [ 1.]]

 ...

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 5.]
  [24.]
  [ 5.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 5.]
  [18.]
  [ 4.]]

 [[ 0.]
  [ 0.]
  [ 0.]
  ...
  [ 5.]
  [ 7.]
  [ 6.]]]


In [ ]:
print()

28


In [ ]:
etpmodel.fit(tokenized_words, tokenized_phonetics, epochs=3, validation_split=0.1)

Epoch 1/3
3802/3802 [==============================] - 145s 36ms/step - loss: 217.1068 - acc: 0.0985 - val_loss: 228.2661 - val_acc: 0.1805
Epoch 2/3
3802/3802 [==============================] - 135s 36ms/step - loss: 223.2668 - acc: 0.1957 - val_loss: 246.7397 - val_acc: 0.1805
Epoch 3/3
3802/3802 [==============================] - 134s 35ms/step - loss: 246.0293 - acc: 0.1957 - val_loss: 271.9679 - val_acc: 0.1805


In [ ]:
word="balls"
tokenized_word_test=np.array(letter_tokenizer.texts_to_sequences([list(word)]))
print(tokenized_word_test)

[[17  2  9  9  5]]


In [ ]:
print(etpmodel(tokenized_word_test.reshape(tokenized_word_test.shape[0], tokenized_word_test.shape[1], 1)))

tf.Tensor(
[[1.17963425e-26 4.67159884e-26 2.56787429e-26 2.14984857e-26
  3.22911522e-26 1.49747339e-26 2.03534264e-26 2.97221404e-26
  2.19658689e-26 4.62203712e-26 1.21531270e-25 1.12930615e-24
  1.66428336e-23 9.15431353e-22 9.30941438e-20 1.69180114e-16
  5.07423747e-13 1.65561387e-09 6.24991969e-07 2.17812922e-04
  3.88885126e-03 8.19307044e-02 7.55403265e-02 9.99388546e-02
  1.32787436e-01 3.24861586e-01 1.51023388e-01 1.29810423e-01]], shape=(1, 28), dtype=float32)
